In [2]:
# Configuration related preprocessing step before mounting the drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.23-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [3]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
#3 set directory path for dataset
import os
os.chdir("/content/drive/My Drive/Mask_Object_Detection")
Dataset='Dataset'
Data_dir=os.listdir(Dataset)
print(Data_dir)

['with_mask', 'without_mask']


In [5]:
# Import necessary libraries

import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

img_rows,img_cols=112,112

images=[]
labels=[]

# to access path of files
for category in Data_dir:
  folder_path=os.path.join(Dataset,category)
  # to access images inside the above path
  for img in os.listdir(folder_path):
    img_path=os.path.join(folder_path,img)
    img=cv2.imread(img_path)
    print(img.shape)

    try:
      # convert the image into greyscale
      grayscale_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

      # rescaling the gray image into size of 112*112 in order to keep all the image size consistent
      resized_img=cv2.resize(grayscale_img,(img_rows,img_cols))
      images.append(resized_img)
      labels.append(category)
    
    # Handle exception in case of any error occurs
    except Exception as e:
      print("Exception : ",e)

# normalizing the image
images=np.array(images)/255.0
#reshaping the image array without changing the dta of image because we want data in specific format
images=np.reshape(images,(images.shape[0],img_rows,img_cols,1))


# perform one hot encoding on labels since labels are in text format

lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)
labels=np.array(labels)

(train_X,text_X,train_y,test_y)=train_test_split(images,labels,test_size=0.25,random_state=0)

Dataset/with_mask/119-with-mask.jpg
Dataset/with_mask/196-with-mask.jpg
Dataset/with_mask/147-with-mask.jpg
Dataset/with_mask/207-with-mask.jpg
Dataset/with_mask/317-with-mask.jpg
Dataset/with_mask/212-with-mask.jpg
Dataset/with_mask/336-with-mask.jpg
Dataset/with_mask/254-with-mask.jpg
Dataset/with_mask/305-with-mask.jpg
Dataset/with_mask/104-with-mask.jpg
Dataset/with_mask/391-with-mask.jpg
Dataset/with_mask/38-with-mask.jpg
Dataset/with_mask/342-with-mask.jpg
Dataset/with_mask/232-with-mask.jpg
Dataset/with_mask/26-with-mask.jpg
Dataset/with_mask/287-with-mask.jpg
Dataset/with_mask/301-with-mask.jpg
Dataset/with_mask/415-with-mask.jpg
Dataset/with_mask/4-with-mask.jpg
Dataset/with_mask/277-with-mask.jpg
Dataset/with_mask/228-with-mask.jpg
Dataset/with_mask/59-with-mask.jpg
Dataset/with_mask/395-with-mask.jpg
Dataset/with_mask/40-with-mask.jpg
Dataset/with_mask/318-with-mask.jpg
Dataset/with_mask/283-with-mask.jpg
Dataset/with_mask/70-with-mask.jpg
Dataset/with_mask/158-with-mask.jpg

## Build Convolution Neural Network (CNN) classification model
